In [33]:
print('sss')

sss


In [34]:
import pandas as pd
import numpy  as np
import glob
from datetime import date, timedelta, datetime
import re

In [35]:
## read pardot
pardot= pd.read_csv(r'\\DESKTOP-KJM0HRO\Shared_Folder\Automation\STEP_Lead_Allocation\Data\2022-04-22\formExport.csv')

##template file
df_template= pd.read_excel(r'C:\Users\arvinmvn\Desktop\sssaudit\STEP_AUDIT_MANUAL_TEMPLATE.xlsx',)
## eligble sellers
Eligble_Seller= pd.read_excel(r'\\DESKTOP-KJM0HRO\Shared_Folder\Automation\STEP_Lead_Allocation\Mappings\STEP_Eligible_Sellers.xlsx',)
## provider ids
SP_ID_Map=pd.read_excel(r'\\DESKTOP-KJM0HRO\Shared_Folder\Automation\STEP_Lead_Allocation\Mappings\Providers Mappingv2.xlsx',)
## sp master

SP_master=pd.read_excel(r'\\DESKTOP-KJM0HRO\Shared_Folder\SP_Master\SPMaster_final.xlsx',)

## aj

aj=pd.read_csv(r'\\DESKTOP-KJM0HRO\Shared_Folder\Automation\Step Allocation Audt\2022-04-25\AJ latest.csv')

C:\Users\arvinmvn\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (15,18,19,23,25,28,30,41,42,43,45,49,52,53,54,56,62,63,65,68,69,75,97,98,102,104,105,108,111,112,113,114,115,121,125,127,138,140,141,154,160,167,176,177,178,179,180,181,183,184,185,186,187,188,190,191,192,194,195,197,198,201,205,213,214,216,220,225,226,232,258,260,261,262,263,264,272,273,274,275,276,277,278,279,280,281,282,283,284,285,287,288,289,290,291,292,295,296,300,301,302,303,304,305,307,308,309,310,319,320,321,322,323,324,325,326,327,328,330,332,333,334,335,336,337,338,339,340,341,343,344,345,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,365,366,367,376,391,403,420,430,431) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [36]:
## take the needed fields

pardot = pardot[['Prospect Id', 'Last Activity At', 'Do you agree to all the terms and conditions?',
                    'STEP_SPN_Merchant', 'STEP_SPN_Name', 'Email', 'STEP_SPN_Contact', 'STEP_SPM_AM',
                    'STEP_SPN_Accounting', 'STEP_SPN_Imaging', 'STEP_SPN_Cataloging', 'STEP_SPN_Advertisement',
                    'STEP_SPN_Brand', 'STEP_SPN_Training', 'Select one of the services for Aplus', 'Select one of the SPN services']]

In [37]:
## clean brand videos name

pardot['Select one of the SPN services']= pardot['Select one of the SPN services'].replace('Brand Videos(Only sellers registered under Amazon Brand Registry are eligible to upload Product Videos)','Brand Videos')

pardot['Select one of the SPN services']=pardot['Select one of the SPN services'].replace('Advertising Management','Advertising Optimization')

pardot['Select one of the SPN services'].unique()       
       

array(['Advertising Optimization', 'Account Management', 'Training',
       'Cataloging', 'Brand Videos', 'I do not want SPN services',
       'Accounting', 'Imaging'], dtype=object)

In [38]:
## MCID encryption function

import sys
import re

"""
Provides functions to obfuscate 64-bit numbers into strings and to de-obfuscate them back.

The obfuscation function maps unsigned 64-bit integers to base36 encoded strings,
and the de-obfuscation function maps base36 strings to unsigned 64-bit integers
using the Tiny Encryption Algorithm designed by David Wheeler and Roger Needham.

This code uses the same basic algorithm as in the original implementations:
- IDs.pm from package Amazon_Crypt_IDs
- amzIDs/amzuid.cpp and amzIDs/simple-cipher.cpp from package APLCore

The obfuscation method for numbers which are less than 2^43 is slightly
different from the one for numbers greater than or equal to 2^43.
In this context a 64-bit number < 2^43 is called a small 64-bit number,
and a 64-bit number >= 2^43 is called a large 64-bit number.

API:

encrypt_id(tag, num)
    obfuscate a unsigned 64-bit integer num automatically detecting whether num is
    a small 64-bit number or a large one, and add tag in front of it
encode_small_num(num)
    obfuscate a 64-bit number num by the method specific to small 64-bit numbers
encode_large_num(num)
    obfuscate a 64-bit number num by the method specific to large 64-bit numbers
decrypt_id(id_str)
    remove the tag in front of id_str and de-obfuscate the rest as a 64-bit number
decode_small_id(token)
    de-obfuscate token as a small 64-bit number
decode_large_id(token)
    de-obfuscate token as a large 64-bit number

References:

- https://code.amazon.com/packages/Amazon_Crypt_IDs/blobs/mainline/--/perlModules/lib/Amazon/Crypt/IDs.pm
- https://code.amazon.com/packages/APLCore/blobs/mainline/--/amzIDs/amzuid.cpp
- https://code.amazon.com/packages/APLCore/blobs/mainline/--/amzIDs/simple-cipher.cpp
- https://w.amazon.com/index.php/Obfuscated_Id
- http://en.wikipedia.org/wiki/Tiny_Encryption_Algorithm

"""

KNOWN_TAGS = {
    'A'   : 'customer',
    'B'   : 'batch',
    'C'   : 'chargeback claim',
    'J'   : 'fppurchase',
    'P'   : 'paypage',
    'R'   : 'reviews',
    'RA'  : 'reusecustomer',
    'RC'  : 'registryclaim',
    'ROC' : 'rmtoneclick',
    'S'   : 'storefront',
    'T'   : 'uploadedexch',
    'x'   : 'forums',
}

MASK_20_BITS = 0xfffff
MASK_32_BITS = 0xffffffff
CIPHER_KEY = (0x7A616D41, 0x632E6E6F, 0x42206D6F, 0x736B6F6F)
DELTA = 0x9E3779B9
MIN_LARGE_NUM = 1<<43  # 2^43
MAX_LARGE_NUM = 1<<64  # 2^64
LARGE_NUM_CHECKSUM_LENGTH = 7
MAX_SMALL_ID_LENGTH = 13

class AmznCustIdEncryptionError(Exception):
    pass

class AmznCustIdDecryptionError(Exception):
    pass

class AmznShipmentIdEncryptionError(Exception):
    pass

def encrypt_id(tag,num):
    """
    The main function to encode a number.
    tag := 'A' | 'P' | 'S' | 'B' | 'RA' | 'RC' | 'ROC' | 'J' | 'T' | 'x' | 'R'
    num := an unsigned 64-bit integer
    """

    if tag not in KNOWN_TAGS:
        raise AmznCustIdEncryptionError(
            "tag argument should be one of %s" % ', '.join(KNOWN_TAGS.keys())
        )
    num = int(num)
    if not (isinstance(num,int) or isinstance(num,long)):
        raise AmznCustIdEncryptionError(
            "num should be of integer type, not %s" % type(num)
        )
    elif num < 0:
        return None
    elif num > MAX_LARGE_NUM:
        return None
    if num < MIN_LARGE_NUM:
        token = encode_small_num(num)
    else:
        token = encode_large_num(num)
    return tag + token


def decrypt_id(id_str):
    '''
    The main function to decode an obfuscated ID.
    obfuscated-id := tag obfuscated-token
    tag := 'A' | 'P' | 'S' | 'B' | 'R' | 'RA' | 'RC' | 'ROC' | 'J' | 'T' | 'x'
    '''
    for tag in KNOWN_TAGS.keys():
        if id_str.startswith(tag):
            token = id_str[len(tag):]
            if len(token) > MAX_SMALL_ID_LENGTH:
                # if obfuscated-token is valid it has to come from a large number
                return decode_large_id(token)
            else:
                # first try decoding it assuming it originates from a small 64-bit number,
                # which would be the case most of the time.
                try:
                    num = decode_small_id(token)
                except (ValueError) as ex:
                    # then try as a large number
                    try:
                        num = decode_large_id(token)
                    except (ValueError) as ex:
                        return None
                    return num
                return num

    return None


def encode_small_num(num):
    '''
    Encode a 64-bit number smaller than 2^43 to generate its obfuscated token.

    They are called small 64-bit numbers.
    Left shift num by 20 bits and add 20-bit checksum of num,
    then obfuscate the result by TEA encryption with the fix cipher CIPHER_KEY.

    This function itself is capable of handling any number less than 2^44
    because we have total 64 bits, and decoding its result back to the original
    with decode_small_id() works.

    However amzIDs/amzuid.cpp in APLCore uses the encoding method in this function only
    for numbers less than 2^43, so we do the same in the current code.
    '''
    num_and_checksum = num << 20
    num_and_checksum += compute_20_bit_checksum(num)
    return disguise_as_alphanum(num_and_checksum)


def encode_large_num(num):
    '''
    Encode a 64-bit number greater than or equal to 2^43 to generate its obfuscated token.

    They are called large 64-bit numbers.
    Add the 20-bits checksum of num in front of disguise_as_alphanum(num).
    The 20-bit checksum is less than 1<<20 which can be written with at most 7 decimals.
    '''
    format_string = '%s%d%s%s' % (
        '%0',                       # 0 padding
        LARGE_NUM_CHECKSUM_LENGTH,  # width of checksum part
        'd',                        # checksum in decimal
        '%s',                       # disguise_as_alphanum(num) in string
    )
    token = format_string % (compute_20_bit_checksum(num),
                             disguise_as_alphanum(num))
    return token


def disguise_as_alphanum(num):
    '''
    Break the given 64 bit number into two 32 bit numbers,
    obfuscate by TEA encryption with the fix cipher CIPHER_KEY and salt DELTA,
    concatenate the resulting two 32-bit numbers to generate one 64-bit number,
    then encode the 64-bit number by base36 encoding.
    The length of generated string will be at most 13 because
    math.log(2**64,36) = 12.37928983150533.
    '''
    val = (num & MASK_32_BITS, num >> 32)
    obfuscated = simple_encode(val, CIPHER_KEY, DELTA)
    obfuscated_data = obfuscated[0] + (obfuscated[1] << 32)
    return base36encode(obfuscated_data)


def compute_20_bit_checksum(num):
    ID_XOR = 0xC2F93;
    checksum = 0
    for ii in range(4):
        checksum += (num & MASK_20_BITS)
        num = num >> 16
    checksum ^= ID_XOR
    checksum &= MASK_20_BITS
    return checksum

def decode_small_id(token):
    num_and_checksum = reveal_number(token)
    checksum = num_and_checksum & MASK_20_BITS
    num = num_and_checksum >> 20
    if checksum == compute_20_bit_checksum(num):
        return num
    else:
        raise ValueError('invalid checksum')

def decode_large_id(token):
    try:
        checksum = int(token[:LARGE_NUM_CHECKSUM_LENGTH])
    except (ValueError) as ex:
        raise ValueError("first %d characters of the token %s are not numerals"
                         % (LARGE_NUM_CHECKSUM_LENGTH, token))
    encoded_id = token[LARGE_NUM_CHECKSUM_LENGTH:]
    num = reveal_number(encoded_id)
    if checksum == compute_20_bit_checksum(num):
        return num
    else:
        raise ValueError('invalid checksum')

def reveal_number(token):
    '''
    ValueError is raised for an invalid input.
    '''
    obfuscated_data = base36decode(token)
    val = (obfuscated_data & MASK_32_BITS, obfuscated_data >> 32)
    decoded = simple_decode(val, CIPHER_KEY, DELTA)
    return decoded[0] + (decoded[1] << 32)


def simple_encode(val, obfuscator, delta):
    '''
    The encryption function for TEA algorithm.
    Encodes a pair of two unsigned 32-bit integers to generate two encrypted unsigned 32-bit integers.
    '''
    y = val[0]
    z = val[1]
    sum = 0
    for ii in range(32):
        sum = u32_add(sum, delta)
        q = u32_add((z << 4) & MASK_32_BITS, obfuscator[0])
        q ^= u32_add(z, sum)
        q ^= u32_add(z >> 5, obfuscator[1])
        y = u32_add(y, q)
        q = u32_add((y << 4) & MASK_32_BITS, obfuscator[2])
        q ^= u32_add(y, sum)
        q ^= u32_add(y >> 5, obfuscator[3])
        z = u32_add(z, q)
    return (y,z)


def simple_decode(val, obfuscator, delta):
    '''
    The decryption function for TEA algorithm.
    Decodes a pair of two unsigned 32-bit integers to generate two decrypted unsigned 32-bit integers.
    '''
    y = val[0]
    z = val[1]
    sum = (delta << 5) & MASK_32_BITS
    for ii in range(32):
        q = u32_add((y << 4) & MASK_32_BITS, obfuscator[2])
        q ^= u32_add(y, sum)
        q ^= u32_add(y >> 5, obfuscator[3])
        z = u32_sub(z, q)
        q = u32_add((z << 4) & MASK_32_BITS, obfuscator[0])
        q ^= u32_add(z, sum)
        q ^= u32_add(z >> 5, obfuscator[1])
        y = u32_sub(y, q)
        sum = u32_sub(sum, delta)
    return (y,z)


def base36encode(number):
    if number < 0:
        raise ValueError('integer must be positive')

    ALPHABET = '0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    base36 = ''
    while number:
        (number, mod) = divmod(number, 36)
        base36 = ALPHABET[mod] + base36
    return base36


def base36decode(base36):
    '''
    Decode the given string as base36 encoded.
    ValueError will be raised for an invalid input.
    '''
    return int(base36, 36)


def u32_add(a, b):
    # ctypes.c_uint32(a+b).value is a little slower than this.
    return (a + b) & MASK_32_BITS


def u32_sub(a, b):
    return (a - b) & MASK_32_BITS

"""
ShipmentId Encryption
"""

SHIPMENT_ID_ALPHAS = ['q','w','e','r','t','y','u','a','d','f','g','h','x','c','v','b']

SHIPMENT_ID_ALPHAS_REVERSE = [-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,
                              -1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,
                              -1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,
                              -1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,
                              -1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,
                              -1,-1,7,15,13,8,2,9,10,11,-1,-1,-1,-1,-1,-1,-1,-1,0,3,-1,
                              4,6,14,1,12,5]

SHIPMENT_ID_EVEN_CHARACTERS = ['P','q','n','p','R','x','J','r','d','s','G','L','0','f',
                               'C','Y','l','H','c','T','z','h','6','Q','9','W','w','Z',
                               'b','t','V','2','M','B','g','8','D','7','1','j','m','k',
                               '3','F','y','4','N','5','S','v','X','K']

SHIPMENT_ID_EVEN_CHARACTERS_REVERSE = [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1,-1, -1, -1,
                                       -1, -1, -1, -1, -1, -1, -1,-1, -1, -1, -1, -1, -1,
                                       -1, -1, -1, -1,-1, -1, -1, -1, -1, -1, -1, -1, -1,
                                       -1,-1, -1, -1, -1, -1, -1, -1, -1, 12, 38, 31, 42,
                                       45, 47, 22, 37, 35, 24, -1, -1,-1, -1, -1, -1, -1,
                                       -1, 33, 14, 36, -1,43, 10, 17, -1, 6, 51, 11, 32,
                                       46, -1, 0, 23, 4, 48, 19, -1, 30, 25, 50, 15, 27,
                                       -1, -1, -1, -1, -1, -1, -1, 28, 18, 8, -1, 13, 34,
                                       21, -1, 39, 41, 16, 40, 2, -1, 3, 1, 7, 9, 29, -1,
                                       49, 26, 5, 44, 20]

SHIPMENT_ID_ODD_CHARACTERS = ['R','N','k','b','H','4','9','B','w','y','Y','D','F','2','V',
                              'z','v','c','Q','s','L','7','r','1','M','J','3','x','n','T',
                              'P','t','G','d','6','S','0','l','m','K','W','p','C','X','8',
                              'g','5','q','Z','j','f','h']

SHIPMENT_ID_ODD_CHARACTERS_REVERSE =  [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
                                       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
                                       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
                                       -1, -1, -1, -1, -1, -1, -1, -1, -1, 36, 23, 13, 26,
                                       5, 46, 34, 21, 44, 6, -1, -1, -1, -1, -1, -1, -1, -1,
                                       7, 42, 11, -1, 12, 32, 4, -1, 25, 39, 20, 24, 1, -1,
                                       30, 18, 0, 35, 29, -1, 14, 40, 43, 10, 48, -1, -1, -1,
                                       -1, -1, -1, -1, 3, 17, 33, -1, 50, 45, 51, -1, 49, 2,
                                       37, 38, 28, -1, 41, 47, 22, 19, 31, -1, 16, 8, 27, 9, 15]

SHIPMENT_ID_END_CHARACTERS = ['D','U','T','M','X','G','Q','B','P','N','L','F','W','E','R','H',
                              'A','K','Z','C','V','Y','J','I','O','S']

SHIPMENT_ID_END_CHARACTERS_REVERSE = [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
                                      -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
                                      -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
                                      -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
                                      -1, -1, -1, -1, -1, 16, 7, 19, 0, 13, 11, 5, 15, 23, 22, 17,
                                      10, 3, 9, 24, 8, 6, 14, 25, 2, 1, 20, 12, 4, 21, 18, -1, -1,
                                      -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
                                      -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]

SHIPMENT_ID_CHARACTERS_REVERSE_LOOKUP_TABLE = [SHIPMENT_ID_END_CHARACTERS_REVERSE,
                                               SHIPMENT_ID_EVEN_CHARACTERS_REVERSE,
                                               SHIPMENT_ID_EVEN_CHARACTERS_REVERSE,
                                               SHIPMENT_ID_ODD_CHARACTERS_REVERSE,
                                               SHIPMENT_ID_ODD_CHARACTERS_REVERSE,
                                               SHIPMENT_ID_EVEN_CHARACTERS_REVERSE,
                                               SHIPMENT_ID_EVEN_CHARACTERS_REVERSE,
                                               SHIPMENT_ID_ODD_CHARACTERS_REVERSE,
                                               SHIPMENT_ID_ODD_CHARACTERS_REVERSE]

SHIPMENT_ID_DECODING_TABLE = [0, 8, 6, 4, 2, 7, 5, 3, 1]
SIZE_OF_ENCRYPTED_SHIPMENT_ID = 9
SHIPMENT_ID_ENCRYPTION_BASE = 52
NIBBLES_IN_SHIPMENT_ID = 16
MAX_SMALL_SHIPMENT_ID = 4839182330
SMALL_SHIPMENT_ID_PATTERN = re.compile("^[qwertyuadfghxcvb]+\d+$")
ENCRYPTED_RMA_PATTERN = re.compile("^(\w+RMA)$")

def encrypt_small_shipment_id(id):
    num = int(id)
    
    if num < 0:
        raise AmznShipmentIdEncryptionError("ShipmentIds are positive. Input: %s" % num)

    if num > MAX_SMALL_SHIPMENT_ID:
        raise AmznShipmentIdEncryptionError("Cannot encrypt a ShipmentId that large. Input: %s" % num)

    nibbles = []
    while (num > 0):
        nibble = num % 16
        nibbles.append(nibble)
        num = num / 16

    numbles = []
    chomped = ""
    for i, nibble in enumerate(nibbles):
        if (i % 2) > 0:
            numbles.append(nibble)
        else:
            alpha = nibbles[i]
            alpha = SHIPMENT_ID_ALPHAS[alpha]
            chomped += alpha
            
    # Tag the digits onto the alphas
    for numble in reversed(numbles):
        num = num << 4
        num = num | numble

    chomped += str(num)

    return chomped
    
def encrypt_large_shipment_id(id):
    decrypted_shipment_id = int(id)
    if decrypted_shipment_id < 0:
        raise AmznShipmentIdEncryptionError("ShipmentId should be positive. Input: %s" % decrypted_shipment_id)

    if decrypted_shipment_id < MAX_SMALL_SHIPMENT_ID:
        raise AmznShipmentIdEncryptionError("ShipmentId too small. Input: %s" % decrypted_shipment_id)

    remainder = 0
    index_to_update = 0
    encrypted_shipment_id = [0] * SIZE_OF_ENCRYPTED_SHIPMENT_ID
    for count in range(SIZE_OF_ENCRYPTED_SHIPMENT_ID-1):
        decrypted_shipment_id, remainder = divmod(decrypted_shipment_id,SHIPMENT_ID_ENCRYPTION_BASE)

        if (count < 4):
            index_to_update = count * 2 + 1
        else:
            index_to_update = (count - 3) * 2

        if (count % 2) > 0:
            encrypted_shipment_id[index_to_update] = SHIPMENT_ID_ODD_CHARACTERS[remainder]
        else:
            encrypted_shipment_id[index_to_update] = SHIPMENT_ID_EVEN_CHARACTERS[remainder]
    if decrypted_shipment_id < len(SHIPMENT_ID_END_CHARACTERS):
        encrypted_shipment_id[0] = SHIPMENT_ID_END_CHARACTERS[decrypted_shipment_id]
    else:
        return None

    return "".join(encrypted_shipment_id)

def small_shipment_id_is_encrypted(id):
    return SMALL_SHIPMENT_ID_PATTERN.match(id.strip()) is not None

def is_encrypted_rma_id(id):
    return ENCRYPTED_RMA_PATTERN.match(id.strip()) is not None

def large_shipment_id_is_encrypted(id):
    id = id.strip()
    if (len(id) != SIZE_OF_ENCRYPTED_SHIPMENT_ID):
        return False
    id = [ord(a) for a in list(id)]
    i = 0
    while (i <= SIZE_OF_ENCRYPTED_SHIPMENT_ID - 1):
        char_ord = id[i]
        if (char_ord > 122 or SHIPMENT_ID_CHARACTERS_REVERSE_LOOKUP_TABLE[i][char_ord] < 0):
            return False
        i += 1
    return True

def decrypt_small_shipment_id(encrypted_shipment_id):
    if not encrypted_shipment_id or not small_shipment_id_is_encrypted(encrypted_shipment_id):
        return None
    
    chomped = encrypted_shipment_id
    nibbles = []

    # Pull the letters off the front
    while (chomped[0].isalpha()):
        alpha = chomped[0]
        nibble = 0
        while (nibble <= 15):
            if alpha == SHIPMENT_ID_ALPHAS[nibble]:
                nibbles.append(nibble)
            nibble += 1
        chomped = chomped[1:]

    # Insert the remaining digit's nibbles into the array
    libbles = (len(nibbles) - 1) * 2
    chomped = int(chomped)
    i = 1
    while (chomped > 0 or i < libbles):
        nibbles.insert(i, chomped & 15)
        chomped >>= 4
        i += 2

    # Shift it all back into an int
    decrypted_small_shipment_id = 0
    start_index = len(nibbles) - 1
    i = start_index
    while (i >= 0):
        decrypted_small_shipment_id = decrypted_small_shipment_id * 16
        decrypted_small_shipment_id += nibbles[i]
        i -= 1
    return decrypted_small_shipment_id

def decrypt_large_shipment_id(encrypted_shipment_id):
    if not encrypted_shipment_id or not large_shipment_id_is_encrypted(encrypted_shipment_id):
        return None
    encrypted_shipment_id = [ord(a) for a in list(encrypted_shipment_id)]
    z = 122
    decrypted_shipment_id = 0
    count = 0
    while (count <= (SIZE_OF_ENCRYPTED_SHIPMENT_ID -1)):
        decrypted_shipment_id = decrypted_shipment_id * SHIPMENT_ID_ENCRYPTION_BASE
        char_ord = encrypted_shipment_id[SHIPMENT_ID_DECODING_TABLE[count]]
        if char_ord > z:
            return None

        to_add = SHIPMENT_ID_CHARACTERS_REVERSE_LOOKUP_TABLE[SHIPMENT_ID_DECODING_TABLE[count]][char_ord]
        if (to_add < 0):
            return None
        
        decrypted_shipment_id = decrypted_shipment_id + to_add
        count += 1
    return decrypted_shipment_id

def decrypt_shipment_id(id):
    if not id or id[0] > "z":
        return None
    id = id.strip()
    char_ord = ord(id[0])
    if SHIPMENT_ID_ALPHAS_REVERSE[char_ord] >= 0:
        return decrypt_small_shipment_id(id)
    elif SHIPMENT_ID_END_CHARACTERS_REVERSE[char_ord] >= 0:
        return decrypt_large_shipment_id(id)
    else:
        return None
    
def encrypt_shipment_id(id):
    if not id:
        return None
    num = int(id)
    if num <= MAX_SMALL_SHIPMENT_ID:
        return encrypt_small_shipment_id(id)
    else:
        return encrypt_large_shipment_id(id)

def encrypt_rma_id(id):
    return encrypt_shipment_id(id) + "RMA"

def decrypt_rma_id(id):
    if not id or not is_encrypted_rma_id(id):
        return None
    encrypted_shipment_id = id.strip().replace("RMA","")
    return decrypt_shipment_id(encrypted_shipment_id)
    
    
    



In [39]:
## change number to alpha numeric MCIDs

for i in range (0,len(pardot['STEP_SPN_Merchant'])):

    try:
        pardot['STEP_SPN_Merchant'].values[i]= encrypt_id('A',pardot['STEP_SPN_Merchant'].values[i])
    except:
        pardot['STEP_SPN_Merchant'].values[i]=pardot['STEP_SPN_Merchant'].values[i]
pardot['STEP_SPN_Merchant']

0       A14HPLBH08WQ94
1       A15P3T0KFEQ31G
2        ARRZRJUEIYUB4
3         AIQ7WG9HJ87Y
4       A1K6STNRS9H0G8
             ...      
2160     AGXR5W9P401YI
2161     AJEZW63H2OKUM
2162    A1FQT2ER3KOQI5
2163       1.79632E+11
2164     A9JLXBUSG824C
Name: STEP_SPN_Merchant, Length: 2165, dtype: object

In [40]:
# # ## Remove multiple requests from the same seller, as the seller can fill new column with a new emaild

pardot=pardot.drop_duplicates(subset=['STEP_SPN_Merchant'], keep='first')


In [41]:
## let's add service name, cleaned provider name, provider id to the pardot file and do a line item of into 2 with enhaned brand content


pardot_aplus=pardot.copy()

pardot_aplus['Select one of the SPN services']='Enhanced Brand Content'

pardot_times2=pd.concat([pardot_aplus,pardot])



## add amazon commments column based on opted service

pardot_times2['amazon comments']=''

for i in range (0,len(pardot_times2['amazon comments'])):
    if pardot_times2['Select one of the SPN services'].values[i]=='Enhanced Brand Content':
        
        pardot_times2['amazon comments'].values[i]='STEP Offer - A+ Cataloguing - Up to 30 ASINs'
        
    if pardot_times2['Select one of the SPN services'].values[i]=='Cataloging':
        
        pardot_times2['amazon comments'].values[i]='STEP Offer - Cataloguing - INR 3500 Credits'
    
    if pardot_times2['Select one of the SPN services'].values[i]=='Cataloguing':
        
        pardot_times2['amazon comments'].values[i]='STEP Offer - Cataloguing - INR 3500 Credits'    
        
    if pardot_times2['Select one of the SPN services'].values[i]=='Account Management':
        pardot_times2['amazon comments'].values[i]='STEP Offer - Account Management - INR 3500 Credits' 
        
    if pardot_times2['Select one of the SPN services'].values[i]=='Brand Videos':
        pardot_times2['amazon comments'].values[i]='STEP Offer - Brand Videos - INR 3500 Credits'
        
    if pardot_times2['Select one of the SPN services'].values[i]=='Imaging':
        pardot_times2['amazon comments'].values[i]='STEP Offer - Imaging - INR 3500 Credits'  
        
    if pardot_times2['Select one of the SPN services'].values[i]=='Advertising Optimization':
        pardot_times2['amazon comments'].values[i]='STEP Offer - Advertising Optimization - INR 3500 Credits' 
        
    if pardot_times2['Select one of the SPN services'].values[i]=='Training':
        pardot_times2['amazon comments'].values[i]='STEP Offer - Training - INR 3500 Credits'     
        
    if pardot_times2['Select one of the SPN services'].values[i]=='Accounting':
        pardot_times2['amazon comments'].values[i]='STEP Offer - Accounting - INR 3500 Credits'
        
        
    if pardot_times2['Select one of the SPN services'].values[i]=='I do not want SPN services':
        pardot_times2['amazon comments'].values[i]='-'    


        
        
        
        
## clean the service provider name

pardot_times2['Service Provider']=''
for i in range(0,len(pardot_times2['Service Provider'])):
    
    if pardot_times2['Select one of the SPN services'].values[i]=='Advertising Optimization':
        pardot_times2['Service Provider'].values[i]=pardot_times2['STEP_SPN_Advertisement'].values[i]
        
    elif pardot_times2['Select one of the SPN services'].values[i]=='Account Management':
        pardot_times2['Service Provider'].values[i]=pardot_times2['STEP_SPM_AM'].values[i]
        
    elif pardot_times2['Select one of the SPN services'].values[i]=='Imaging':
        pardot_times2['Service Provider'].values[i]=pardot_times2['STEP_SPN_Imaging'].values[i]
        
        
    elif pardot_times2['Select one of the SPN services'].values[i]=='Training':
        pardot_times2['Service Provider'].values[i]=pardot_times2['STEP_SPN_Training'].values[i]
        
        
    elif pardot_times2['Select one of the SPN services'].values[i]=='Brand Videos':
        pardot_times2['Service Provider'].values[i]=pardot_times2['STEP_SPN_Brand'].values[i] 
        
        
    elif pardot_times2['Select one of the SPN services'].values[i]=='Accounting':
        pardot_times2['Service Provider'].values[i]=pardot_times2['STEP_SPN_Accounting'].values[i]     
        
    
    elif pardot_times2['Select one of the SPN services'].values[i]=='Cataloguing':
        pardot_times2['Service Provider'].values[i]=pardot_times2['STEP_SPN_Cataloging'].values[i]
        
        
    elif pardot_times2['Select one of the SPN services'].values[i]=='Cataloging':
        pardot_times2['Service Provider'].values[i]=pardot_times2['STEP_SPN_Cataloging'].values[i] 
        
        
    elif pardot_times2['Select one of the SPN services'].values[i]=='I do not want SPN services':
        pardot_times2['Service Provider'].values[i]='-'
        
        
    elif pardot_times2['Select one of the SPN services'].values[i]=='Enhanced Brand Content':
        pardot_times2['Service Provider'].values[i]=pardot_times2['Select one of the services for Aplus'].values[i]    
    

   
    pardot_times2['Service Provider'].values[i]=re.sub("\(.*?\)","()",pardot_times2['Service Provider'].astype(str).values[i]).strip("()").strip()


    
## Add the SP IDs based on the Service Provider Name





pardot_times2 =pd.merge(pardot_times2,SP_ID_Map[['SP Name','Provider_Id']], left_on=['Service Provider'],right_on=['SP Name'],how='left')

## Add SP and provider_id as - for all the I do not want A+ cataloging  (the one for main services is already done in earlier steps)

## Add SP and provider_id as - for all the I do not want SPN service  (the one for enhanced brand content services is already done in earlier steps)


for i in range (0,len(pardot_times2)):
    
    if pardot_times2['Service Provider'].values[i]=='-':
        pardot_times2['SP Name'].values[i]='-'
        pardot_times2['Provider_Id'].values[i]='-' 
        
    elif pardot_times2['Service Provider'].values[i]=='I do not want A+ service':
        pardot_times2['SP Name'].values[i]='-'
        pardot_times2['Provider_Id'].values[i]='-'   

pardot_times2['Provider_Id']=pardot_times2['Provider_Id']. fillna('Provider ID not found')
pardot_times2['SP Name']=pardot_times2['SP Name']. fillna('SP not found')


pardot_times2=pardot_times2.drop_duplicates()
print('The above df pardot_times2 gives 1 is to 2 services (adding ebc as complimentry) with amazon comments, service provider name and provider id')

The above df pardot_times2 gives 1 is to 2 services (adding ebc as complimentry) with amazon comments, service provider name and provider id


In [43]:
print('pardot time 2 before the multiple id instances (because of multiple email id check)',len(pardot_times2))

pardot time 2 before the multiple id instances (because of multiple email id check) 4276


In [44]:
len(pardot_times2)

4276

In [45]:
## correct phone numbers function
def correct_phone(df, phone_col, reduce_ten_digit):
    df[phone_col].fillna('0', inplace=True)
    # line 1:5 removing trailing 0 from phone strings
    df[phone_col] = df[phone_col].astype('str')
    for phone, ind in zip(df[phone_col].values, df.index):
        if '.0' in phone[-2:]:
            df.loc[df.index == ind, phone_col] = phone[:-2]

    df[phone_col] = df[phone_col].str.replace('[^0-9]', '', regex=True)
    if reduce_ten_digit:
        df[phone_col] = df[phone_col].str[-10:]
    df[phone_col] = df[phone_col].replace('', '0')
    df[phone_col] = df[phone_col].astype(np.int64)
    df[phone_col] = df[phone_col].astype(str)

    return df

correct_phone(pardot_times2, 'STEP_SPN_Contact', True)
pardot_times2['STEP_SPN_Contact'].unique()

array(['7506170304', '9230038077', '9514403131', ..., '6397993899',
       '7092995758', '9887133802'], dtype=object)

In [13]:
#pardot_times2.to_excel(r"C:\Users\arvinmvn\Desktop\test.xlsx",index=False)

In [46]:
print('Allocated step leads in AJ',len(aj))

Allocated step leads in AJ 3751


In [47]:
## total step responses time 2
print('Total STEP Responses Received :-',len(pardot_times2))

Total STEP Responses Received :- 4276


In [48]:
##   of Sellers Not interested in either service (remove both A+ and main)

not_interested=pardot_times2[(pardot_times2['Select one of the services for Aplus'] == 'I do not want A+ service') & (pardot_times2['Select one of the SPN services']=='I do not want SPN services' )]
interested=pardot_times2[(pardot_times2['Select one of the services for Aplus'] != 'I do not want A+ service') | (pardot_times2['Select one of the SPN services']!='I do not want SPN services' )]


print ('#of Sellers Not interested in either service ',len(not_interested))
print ('#of Sellers interested in either service ',len(interested))

#of Sellers Not interested in either service  13
#of Sellers interested in either service  4263


In [49]:
## correct n Wrong MCID cases
valid_mcids=interested[interested['STEP_SPN_Merchant'].str.match('A')]
invalid_mcids=interested[~interested['STEP_SPN_Merchant'].str.match('A')]
print('incorrect MCIDs',len(invalid_mcids))
print('correct MCIDs',len(valid_mcids))

incorrect MCIDs 154
correct MCIDs 4109


In [50]:
## elgible and ineligble sellers

valid_mcids=pd.merge(valid_mcids,Eligble_Seller['encrypted_sellerid'], left_on="STEP_SPN_Merchant",right_on='encrypted_sellerid',how='left')

ineligble_sellers = valid_mcids[valid_mcids['encrypted_sellerid'].isna()]

eligble_sellers=valid_mcids[valid_mcids['encrypted_sellerid'].notna()]

print('Inelgible sellers',len(ineligble_sellers))
print('elgible sellers',len(eligble_sellers))

Inelgible sellers 32
elgible sellers 4077


In [51]:
## correct / incorrect phone numbers

incorrect_numbers=eligble_sellers[eligble_sellers['STEP_SPN_Contact'].astype(str).str.len()!=10]
correct_numbers=eligble_sellers[eligble_sellers['STEP_SPN_Contact'].astype(str).str.len()==10]

print('Incorrect Numbers',len(incorrect_numbers))
print('Correct Numbers',len(correct_numbers))

Incorrect Numbers 45
Correct Numbers 4032


In [20]:
#len(correct_numbers[(correct_numbers['Select one of the services for Aplus']=='I do not want A+ cataloging')&(correct_numbers['Select one of the SPN services']=='Enhanced Brand Content')])

In [52]:
## not interested in SPN

not_interested_Aplus=correct_numbers[(correct_numbers['Select one of the services for Aplus']=='I do not want A+ service')&(correct_numbers['Select one of the SPN services']=='Enhanced Brand Content')]

not_interested_Main=correct_numbers[correct_numbers['Select one of the SPN services']=='I do not want SPN services']


drop_not_interested_Aplus=correct_numbers[(correct_numbers['Select one of the services for Aplus']!='I do not want A+ service')|(correct_numbers['Select one of the SPN services']!='Enhanced Brand Content')]

drop_not_interest_Main=drop_not_interested_Aplus[drop_not_interested_Aplus['Select one of the SPN services']!='I do not want SPN services']

print('Not interested in Aplus',len(not_interested_Aplus))
print('interested in Aplus',len(drop_not_interested_Aplus))
print('Not interested in Main',len(not_interested_Main))
print('interested in Main',len(drop_not_interest_Main))

Not interested in Aplus 203
interested in Aplus 3829
Not interested in Main 48
interested in Main 3781


In [53]:
## 1st not interest A + is dropped, then we are referencing the new result to drop not interested main and dropping the not interest SPN 

valid_requests=drop_not_interest_Main.copy()
len(valid_requests)

3781

In [23]:
#valid_requests.to_excel(r"C:\Users\arvinmvn\Desktop\test.xlsx",index=False)

In [54]:
## aj clean up

## filter the aj file, take only requestsource as 'Referral' and requestsourcereftag as 'Step'

# aj=aj[aj['requestsource']=='Referral']

# aj=aj[aj['requestsourcereftag']=='STEP Program']


## rename Advertisement Optimization and Cataloging to match the service names for req_list df

aj['servic']=aj['service'].replace('Advertising Optimization','Advertisement Optimization')
aj['service']=aj['service'].replace('Cataloguing','Cataloging')


In [55]:
valid_requests.columns

Index(['Prospect Id', 'Last Activity At',
       'Do you agree to all the terms and conditions?', 'STEP_SPN_Merchant',
       'STEP_SPN_Name', 'Email', 'STEP_SPN_Contact', 'STEP_SPM_AM',
       'STEP_SPN_Accounting', 'STEP_SPN_Imaging', 'STEP_SPN_Cataloging',
       'STEP_SPN_Advertisement', 'STEP_SPN_Brand', 'STEP_SPN_Training',
       'Select one of the services for Aplus',
       'Select one of the SPN services', 'amazon comments', 'Service Provider',
       'SP Name', 'Provider_Id', 'encrypted_sellerid'],
      dtype='object')

In [57]:
##Leads unallocated as per All joins

valid_req_lukup_with_aj =pd.merge(valid_requests,aj[['sellercustomerid','amazoncomments']], left_on=['STEP_SPN_Merchant','amazon comments',],right_on=['sellercustomerid','amazoncomments'],how='left')

difference_aj=valid_req_lukup_with_aj[valid_req_lukup_with_aj['sellercustomerid'].isna()].drop_duplicates()

print('Available in pardot and not in aj',len(difference_aj))

Available in pardot and not in aj 235


In [27]:
# len(pardot_times2)

# col_filter=pd.read_excel(r'\\DESKTOP-KJM0HRO\Shared_Folder\Automation\STEP_Lead_Allocation\Data\2022-04-12\col_filtered_download_file.xlsx')

# unalloc_from_Step_alloc=pd.read_excel(r'\\DESKTOP-KJM0HRO\Shared_Folder\Automation\STEP_Lead_Allocation\Data\2022-04-12\Unallocated.xlsx')




# ebc_conf= pd.merge(unalloc_from_Step_alloc,col_filter, left_on=['Seller ID'],right_on=['STEP_SPN_Merchant'],how='left')

# ebc_conf=ebc_conf.rename(columns={'Prospect_Id': 'Prospect Id'})
# ebc_conf=ebc_conf.rename(columns={'Amazon Comments': 'amazon comments'})
# ebc_conf=ebc_conf.rename(columns={'Select one of the services for Aplus': 'amazon comments'})
# ebc_conf=ebc_conf[['Prospect Id', 'Last Activity At', 'Do you agree to all the terms and conditions?',
#                     'STEP_SPN_Merchant', 'STEP_SPN_Name', 'Email', 'STEP_SPN_Contact', 'STEP_SPM_AM',
#                     'STEP_SPN_Accounting', 'STEP_SPN_Imaging', 'STEP_SPN_Cataloging', 'STEP_SPN_Advertisement',
#                     'STEP_SPN_Brand', 'STEP_SPN_Training', 'Select one of the services for Aplus', 'Select one of the SPN services']]

# ebc_conf.rename(columns={'Prospect_Id': 'Prospect Id'})

# ebc_conf.to_excel(r"C:\Users\arvinmvn\Desktop\test_eb.xlsx",index=False)

In [58]:

## we are doing a force stop for all the A+ line items which has same A+ SP and Brand videos SP, these are capture as ebc conflicts in alloc



ebc_conflicts_overall=valid_requests[(valid_requests['Select one of the SPN services']=='Enhanced Brand Content')&(valid_requests['STEP_SPN_Brand']==valid_requests['Select one of the services for Aplus'])]

ebc_conflicts_unalloc= pd.merge(ebc_conflicts_overall,aj[['sellercustomerid','amazoncomments']].rename(columns={'sellercustomerid': 'sellerid'}), left_on=['STEP_SPN_Merchant','amazon comments',],right_on=['sellerid','amazoncomments'],how='left')

# [unallocated_leads['sellerid'].isna()].drop_duplicates()


ebc_conflicts_unalloc=ebc_conflicts_unalloc[ebc_conflicts_unalloc['sellerid'].isna()].drop_duplicates()

len(ebc_conflicts_unalloc)

33

In [59]:
# # # ## EBC conflicts


## we are doing a force stop for all the A+ line items which has same A+ SP and Brand videos SP, these are capture as ebc conflicts in alloc



ebc_conflicts_overall=valid_requests[(valid_requests['Select one of the SPN services']=='Enhanced Brand Content')&(valid_requests['STEP_SPN_Brand']==valid_requests['Select one of the services for Aplus'])]

ebc_conflicts_overalll= pd.merge(ebc_conflicts_overall,aj[['sellercustomerid','amazoncomments']].rename(columns={'sellercustomerid': 'sellerid'}), left_on=['STEP_SPN_Merchant','amazon comments',],right_on=['sellerid','amazoncomments'],how='left')


## we will lookup the total  ebc conflict leads from pardot in AJ, to identify the total unallocated ebc conflict leads

ebc_conflicts=ebc_conflicts_overalll[ebc_conflicts_overalll['sellerid'].isna()].drop_duplicates()



## Next is we will look up the total unalloc leads



## append the list with unalloc leads from pardot times 2 vs aj

unallocated_leads=pd.concat([ebc_conflicts_unalloc,difference_aj]).drop_duplicates()


# ## do a lookup  with AJ to find out what leads didnt get allocated


# unallocated_leads =pd.merge(unallocated_leads_list,aj[['sellercustomerid','amazoncomments']].rename(columns={'sellercustomerid': 'sellerid'}), left_on=['STEP_SPN_Merchant','amazon comments',],right_on=['sellerid','amazoncomments'],how='left')

# unallocated_leads=unallocated_leads[unallocated_leads['sellerid'].isna()].drop_duplicates()


print('Overall EBC Conflicts',len(ebc_conflicts_overall))
print('Unallocated EBC conflicts',len(ebc_conflicts))
print('Total unallocated leads plus leads with ebc conflicts',len(unallocated_leads))

Overall EBC Conflicts 267
Unallocated EBC conflicts 33
Total unallocated leads plus leads with ebc conflicts 235


In [30]:
#difference_aj.drop(['providerid', 'amazoncomments','encrypted_sellerid'], axis=1).to_excel(r"C:\Users\arvinmvn\Desktop\test.xlsx",index=False)

#difference_aj=difference_aj.drop(['sellercustomerid', 'amazoncomments','encrypted_sellerid'], axis=1)

In [60]:
## keep only active sellers




active_sp_aj = pd.merge(unallocated_leads[unallocated_leads['Provider_Id']!='Provider ID not found'],SP_master['Provider_ID'], left_on=['Provider_Id'],right_on=['Provider_ID'],how='left').drop_duplicates()

active_final= active_sp_aj[active_sp_aj['Provider_ID'].notna()]
suspended_final= active_sp_aj[active_sp_aj['Provider_ID'].isna()]

print('the active sp are ',len(active_final))
print('the inactive sp are ',len(suspended_final))


the active sp are  221
the inactive sp are  2


In [61]:
# Invalid Service/SP//subservice

# let's lookup on service and provider id, using the provider mapping file, this will help us track the invalid service combos for active providers
, 
# first clean the similar services advertising service and cataloguing



#SP_ID_Map['Service']=SP_ID_Map['Service'].replace('Advertising Optimization','Advertisement Optimization')
#SP_ID_Map['Service']=SP_ID_Map['Service'].replace('Cataloguing','Cataloging')

invalid_service_subservice_provider_id =pd.merge(active_final,SP_ID_Map[['Service','Provider_Id']].rename(columns={'Provider_Id': 'Provider_Ids'}), left_on=['Provider_Id','Select one of the SPN services'],right_on=['Provider_Ids','Service'],how='left').drop_duplicates()


invalid_service_combo=invalid_service_subservice_provider_id[invalid_service_subservice_provider_id['Provider_Ids'].isna()]

unallocated_leads_with_valid_service_Provider_id=invalid_service_subservice_provider_id[invalid_service_subservice_provider_id['Provider_Ids'].notna()]


print(' with invalid service/subservice/providerid',len(invalid_service_combo))
print(' with valid service/subservice/providerid',len(unallocated_leads_with_valid_service_Provider_id))
print('the number is less than total diff of aj because we removed provider id not found')

 with invalid service/subservice/providerid 0
 with valid service/subservice/providerid 221
the number is less than total diff of aj because we removed provider id not found


In [62]:
# Removing already availed main/a plus service

## prepare a flag for main service and A+ based on the selected option by seller in pardot

unallocated_leads_with_valid_service_Provider_id['flag1']=''

for i in range(0,len(unallocated_leads_with_valid_service_Provider_id)):
    if  unallocated_leads_with_valid_service_Provider_id['Select one of the SPN services'].values[i]!='Enhanced Brand Content':
        unallocated_leads_with_valid_service_Provider_id['flag1'].values[i] = 'Main'
        
    if  unallocated_leads_with_valid_service_Provider_id['Select one of the SPN services'].values[i]=='Enhanced Brand Content':
        unallocated_leads_with_valid_service_Provider_id['flag1'].values[i] = 'Aplus'

        
## prepare a flag for main service and A+ allocated in AJ        
        
already_alloc=aj[['sellercustomerid','amazoncomments']].drop_duplicates()        
already_alloc['flag2']=''


for i in range(0,len(already_alloc)):
    if  already_alloc['amazoncomments'].values[i]!='STEP Offer - A+ Cataloguing - Up to 30 ASINs':
        already_alloc['flag2'].values[i] = 'Main'
        
    if  already_alloc['amazoncomments'].values[i]=='STEP Offer - A+ Cataloguing - Up to 30 ASINs':
        already_alloc['flag2'].values[i] = 'Aplus'
        
        
        
        
avail_r_not = pd.merge(unallocated_leads_with_valid_service_Provider_id, already_alloc[['sellercustomerid','amazoncomments' ,'flag2']],left_on=['STEP_SPN_Merchant','flag1'],right_on=['sellercustomerid','flag2'], how='left')
not_already_availed = avail_r_not[avail_r_not['flag2'].isna()].drop_duplicates()
already_availed = avail_r_not[avail_r_not['flag2'].notna()].drop_duplicates()

print('The seller who have not availed A+/main service',len(not_already_availed))
print('The seller who have already availed A+/main service',len(already_availed))
#not_already_availed = not_already_availed.drop(columns=['flag1'])


The seller who have not availed A+/main service 148
The seller who have already availed A+/main service 73


In [34]:
# #avail_r_not.to_excel(r"C:\Users\arvinmvn\Desktop\test.xlsx",index=False)
# already_alloc['flag2'].unique()

In [63]:
## Report View

df = pd.DataFrame(columns=['Total STEP Responses Received',
                           '# of Sellers Not interested in either service',
                           'Wrong MCID cases',
                           '# of inelligible sellers',
                           '# of sellers with wrong phone numbers',
                           '# of sellers not interested in SPN credits',
                           '# of sellers not interested in A+ benefit',
                           'Valid Requests for which orders need to be created',
                           'Leads allocated as per All joins',
                           'Unallocated leads',
                           'SPid not found not in aj',
                           'Suspended SPs',
                           'EBC Conflict',
                           'Invalid service/service provider/sub service',
                           'Already availed A+/Main Service'
                    
                           ])

df = df.append({'Total STEP Responses Received':len(pardot_times2), 
                '# of Sellers Not interested in either service': len(not_interested),
                'Wrong MCID cases':len(invalid_mcids),
                '# of inelligible sellers':len(ineligble_sellers),
                '# of sellers with wrong phone numbers':len(incorrect_numbers),
                '# of sellers not interested in SPN credits':len(not_interested_Main),
                '# of sellers not interested in A+ benefit':len(not_interested_Aplus),
                'Valid Requests for which orders need to be created':len(valid_requests),
                'Leads allocated as per All joins':len(aj),
                'Unallocated leads':len(unallocated_leads),
                'SPid not found not in aj':len(difference_aj[difference_aj['Provider_Id']=='Provider ID not found']),
                'Suspended SPs':len(suspended_final),
                'EBC Conflict':len(ebc_conflicts),
                'Invalid service/service provider/sub service':len(invalid_service_combo),
                'Already availed A+/Main Service':len(already_availed)
                }, ignore_index=True)
df_audit_report=df.transpose()
df_audit_report
df_audit_report=df_audit_report.rename(columns={df_audit_report.columns[0]: 'Scale'})

df_audit_report['Description']=''
df_audit_report['Description'][0]='Total unique seller requests into 2'
df_audit_report['Description'][1]='Sellers not interested in SPN and A+ service into 2'
df_audit_report['Description'][2]='MCID not starting with A'
df_audit_report['Description'][3]='Seller not availble in the eligble step sellers repository file'
df_audit_report['Description'][4]='Phone number provided not equal to 10 digits'
df_audit_report['Description'][5]='Sellers not interested in SPN service'
df_audit_report['Description'][6]='Sellers not interested in A+ service'
df_audit_report['Description'][7]='total response - all the breakdown above'
df_audit_report['Description'][8]='Leads available in AJ'
df_audit_report['Description'][9]='Unallocate leads based on todays AJ plus day minus ebc videos 2 pardot'
df_audit_report['Description'][10]='Provider ID not found'
df_audit_report['Description'][11]='List of Suspended Provider IDs'
df_audit_report['Description'][12]='day minus 2 ebc conflicts derived from step allocation, we will append this with AJ and do a drop duplicates'
df_audit_report['Description'][13]='the services/subservice provider cant give'
df_audit_report['Description'][14]='Seller has already availed service for Main/A+'
df_audit_report

,Scale,Description
Total STEP Responses Received,4276,Total unique seller requests into 2
# of Sellers Not interested in either service,13,Sellers not interested in SPN and A+ service i...
Wrong MCID cases,154,MCID not starting with A
# of inelligible sellers,32,Seller not availble in the eligble step seller...
# of sellers with wrong phone numbers,45,Phone number provided not equal to 10 digits
# of sellers not interested in SPN credits,48,Sellers not interested in SPN service
# of sellers not interested in A+ benefit,203,Sellers not interested in A+ service
Valid Requests for which orders need to be created,3781,total response - all the breakdown above
Leads allocated as per All joins,3751,Leads available in AJ
Unallocated leads,235,Unallocate leads based on todays AJ plus day m...


In [64]:
## write all the invalid responses for Mayank's documentation

## Report template

# Create a Pandas Excel writer using XlsxWriter as the engine.
date='2022-04-25'

#writer = pd.ExcelWriter(r'C:\Users\arvinmvn\Desktop\sssaudit\stepauditreport_latest_fina2.xlsx', engine='xlsxwriter')

writer = pd.ExcelWriter(r"\\DESKTOP-KJM0HRO\Shared_Folder\Automation\Step Allocation Audt" + "\\" + date + "\\" + "Step_Audit_View.xlsx",engine='xlsxwriter')

# Write each dataframe to a different worksheet.
df_audit_report.to_excel(writer, sheet_name='Summary',header = True)
pardot.to_excel(writer, sheet_name='pardot_input')
pardot_times2.to_excel(writer, sheet_name='pardot_times2')
# not_interested.to_excel(writer, sheet_name='not_interested')
#invalid_mcids.to_excel(writer, sheet_name='invalid_mcids')
# eligble_sellers.to_excel(writer, sheet_name='eligble_sellers')
#incorrect_numbers.to_excel(writer, sheet_name='incorrect_numbers')
# eligble_sellers.to_excel(writer, sheet_name='eligble_sellers')
# not_interested_Aplus.to_excel(writer, sheet_name='not_interested_Aplus')
# not_interested_Main.to_excel(writer, sheet_name='not_interested_Main')
valid_requests.to_excel(writer, sheet_name='valid_requests')
aj.to_excel(writer, sheet_name='aj')
difference_aj.to_excel(writer, sheet_name='Unallocated Leads')
#invalid_service_combo.to_excel(writer, sheet_name='invalid_service_combo')
#suspended_final.to_excel(writer, sheet_name='suspended SPs')
#already_availed.to_excel(writer, sheet_name='already availed service')
#unallocated_leads_with_valid_service_Provider_id.to_excel(writer, sheet_name='final list')


# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [65]:
## output file push to folder

date='2022-04-25'

pardot.to_excel(r"\\DESKTOP-KJM0HRO\Shared_Folder\Automation\Step Allocation Audt" + "\\" + date + "\\" + "pardot_clean.xlsx",index=False)
pardot_times2.to_excel(r"\\DESKTOP-KJM0HRO\Shared_Folder\Automation\Step Allocation Audt" + "\\" + date + "\\" + "pardot time 2.xlsx",index=False)
not_interested.to_excel(r"\\DESKTOP-KJM0HRO\Shared_Folder\Automation\Step Allocation Audt" + "\\" + date + "\\" + "not interested in both services.xlsx",index=False)
invalid_mcids.to_excel(r"\\DESKTOP-KJM0HRO\Shared_Folder\Automation\Step Allocation Audt" + "\\" + date + "\\" + "invalid mcids.xlsx",index=False)
incorrect_numbers.to_excel(r"\\DESKTOP-KJM0HRO\Shared_Folder\Automation\Step Allocation Audt" + "\\" + date + "\\" + "incorrect numbers.xlsx",index=False)
ineligble_sellers.to_excel(r"\\DESKTOP-KJM0HRO\Shared_Folder\Automation\Step Allocation Audt" + "\\" + date + "\\" + "ineligble sellers.xlsx",index=False)
not_interested_Aplus.to_excel(r"\\DESKTOP-KJM0HRO\Shared_Folder\Automation\Step Allocation Audt" + "\\" + date + "\\" + "not_interested_Aplus.xlsx",index=False)
not_interested_Main.to_excel(r"\\DESKTOP-KJM0HRO\Shared_Folder\Automation\Step Allocation Audt" + "\\" + date + "\\" + "not_interested_Main.xlsx",index=False)
valid_requests.to_excel(r"\\DESKTOP-KJM0HRO\Shared_Folder\Automation\Step Allocation Audt" + "\\" + date + "\\" + "valid requests.xlsx",index=False)
difference_aj.to_excel(r"\\DESKTOP-KJM0HRO\Shared_Folder\Automation\Step Allocation Audt" + "\\" + date + "\\" + "unallocated_leads.xlsx",index=False)
ebc_conflicts.to_excel(r"\\DESKTOP-KJM0HRO\Shared_Folder\Automation\Step Allocation Audt" + "\\" + date + "\\" + "ebc conflict.xlsx",index=False)
invalid_service_combo.to_excel(r"\\DESKTOP-KJM0HRO\Shared_Folder\Automation\Step Allocation Audt" + "\\" + date + "\\" + "invalid_serivce_combo.xlsx",index=False)
suspended_final.to_excel(r"\\DESKTOP-KJM0HRO\Shared_Folder\Automation\Step Allocation Audt" + "\\" + date + "\\" + "suspended_SPs.xlsx",index=False)
already_availed.to_excel(r"\\DESKTOP-KJM0HRO\Shared_Folder\Automation\Step Allocation Audt" + "\\" + date + "\\" + "already availed service.xlsx",index=False)
not_already_availed.to_excel(r"\\DESKTOP-KJM0HRO\Shared_Folder\Automation\Step Allocation Audt" + "\\" + date + "\\" + "reupload.xlsx",index=False)


In [38]:
#ineligble_sellers.to_excel(r"C:\Users\arvinmvn\Desktop\test.xlsx",index=False)

In [39]:
# ## Reupload File for bulk

# date='2022-04-11'
# final_list=not_already_availed.copy()

# reupload_file=pd.DataFrame()

# reupload_file['Parent Request ID'] = ''
# reupload_file['Seller ID'] = final_list['STEP_SPN_Merchant']
# reupload_file['Seller Name'] = final_list['STEP_SPN_Name']
# reupload_file['Seller Contact'] = final_list['STEP_SPN_Contact']
# reupload_file['Provider ID'] = final_list['Provider_Id']
# reupload_file['Service Category'] = final_list['Select one of the SPN services']
# reupload_file['Service Type Requested'] = 'Unknown'


# ## if service is brand videos then change it's main service to EBC and service type request to ebc product videos
# for i in range (0,len(final_list)):
#     if final_list['Select one of the SPN services'].values[i]=='Brand Videos':
#         reupload_file['Service Category'].values[i]='Enhanced Brand Content'
#         reupload_file['Service Type Requested'].values[i]='EBC Product Videos'  


# reupload_file['Request generated Date'] = date
# reupload_file['Request Source'] = 'Referral'
# reupload_file['Request Source sub code'] = 'STEP Program'
# reupload_file['Aegis agent name'] = ''
# reupload_file['Offer Availed'] = ''
# reupload_file['Seller Sell in Country'] = 'India'
# reupload_file['Seller Location'] = 'India'
# reupload_file['Allocation Criterion'] = ''

In [40]:
# print ('Reupload Enhanced Brand Content',len(reupload_file[reupload_file['Service Category']=='Enhanced Brand Content']))

In [41]:
# print('Reupload Brand videos',len(final_list[final_list['Select one of the SPN services']=='Brand Videos']))

In [42]:
#reupload_file.to_excel(r"C:\Users\arvinmvn\Desktop\reupload.xlsx",index=False)